# Webscraping | Day03 | API wrappers


In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import getpass
import pprint

In [2]:
client_id = str(getpass.getpass('client_id?'))
client_secret = str(getpass.getpass('client_secret?'))

client_id?········
client_secret?········


In [3]:
#embedding our getpass credentials in the access key 
sp=spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,client_secret=client_secret))

In [12]:
playlist=sp.user_playlist_tracks('Blizzard','0I77w4YsrOdarNjkTfRFTj')

In [13]:
playlist["items"][3]["track"].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [14]:
len(playlist["items"])

100

In [15]:
#function which handles more than 100 results 

def get_playlist_tracks(user_id,playlist_id):
    results=sp.user_playlist_tracks(user_id,playlist_id)
    tracks=results["items"]
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks 


In [17]:
results=get_playlist_tracks('Blizzard','0I77w4YsrOdarNjkTfRFTj')

In [18]:
len(results)

784

In [46]:
names=[]
ids=[]
artist=[]
playlist=results

for song in range(len(results)):
    names.append(results[song]['track']['name'])
    ids.append(results[song]['track']['id'])
    artist.append(results[song]['track']['artists'][0]['name'])

In [47]:
playlist_song_info=pd.DataFrame({"names":names,"ids":ids,"artist":artist})

In [48]:
playlist_song_info

,names,ids,artist
0,Another One Bites The Dust - Remastered 2011,57JVGBtBLCfHw2muk5416J,Queen
1,Hooked On A Feeling,2Nz6aF1umHh5Et6I5H581L,Blue Swede
2,Paradise City,5Q41NLTmGbVPozwHKK7bk2,Guns N' Roses
3,Don't Stop Believin',4bHsxqR3GMrXTxEPLuK5ue,Journey
4,Separate Ways (Worlds Apart),4ECNtOnqzxutZkXP4TE3n3,Journey
...,...,...,...
779,Penthouse Pauper,4aVs8mfyZjVDivGneIIQ5C,Creedence Clearwater Revival
780,Beast Of Burden - Remastered 1994,77oU2rjC5XbjQfNe3bD6so,The Rolling Stones
781,Strange Days,4WFVX3E568hdAixIj3U222,The Doors
782,867-5309 / Jenny,6iX1f3r7oUJnMbGgQ2gx1j,Tommy Tutone


In [49]:
feats=sp.audio_features(tracks=ids[0:99])

In [50]:
feats[0].keys()

dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [51]:
audio_feat_df=pd.DataFrame(sp.audio_features(tracks=ids[0:99]))

In [52]:
from tqdm.notebook import tqdm

In [53]:
audio_features=[]

for ids in tqdm(range(len(playlist_song_info))):
    audio_features.extend(sp.audio_features(tracks=playlist_song_info['ids'][ids]))


  0%|          | 0/784 [00:00<?, ?it/s]

In [54]:
audio_features

[{'danceability': 0.933,
  'energy': 0.528,
  'key': 5,
  'loudness': -6.472,
  'mode': 0,
  'speechiness': 0.162,
  'acousticness': 0.112,
  'instrumentalness': 0.329,
  'liveness': 0.163,
  'valence': 0.756,
  'tempo': 109.975,
  'type': 'audio_features',
  'id': '57JVGBtBLCfHw2muk5416J',
  'uri': 'spotify:track:57JVGBtBLCfHw2muk5416J',
  'track_href': 'https://api.spotify.com/v1/tracks/57JVGBtBLCfHw2muk5416J',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/57JVGBtBLCfHw2muk5416J',
  'duration_ms': 214653,
  'time_signature': 4},
 {'danceability': 0.547,
  'energy': 0.82,
  'key': 8,
  'loudness': -6.728,
  'mode': 1,
  'speechiness': 0.0805,
  'acousticness': 0.261,
  'instrumentalness': 0,
  'liveness': 0.3,
  'valence': 0.933,
  'tempo': 118.208,
  'type': 'audio_features',
  'id': '2Nz6aF1umHh5Et6I5H581L',
  'uri': 'spotify:track:2Nz6aF1umHh5Et6I5H581L',
  'track_href': 'https://api.spotify.com/v1/tracks/2Nz6aF1umHh5Et6I5H581L',
  'analysis_url': 'https://api.spotif

In [55]:
df2=pd.DataFrame(audio_features)

In [58]:
df2

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.933,0.528,5,-6.472,0,0.1620,0.11200,0.329000,0.1630,0.756,109.975,audio_features,57JVGBtBLCfHw2muk5416J,spotify:track:57JVGBtBLCfHw2muk5416J,https://api.spotify.com/v1/tracks/57JVGBtBLCfH...,https://api.spotify.com/v1/audio-analysis/57JV...,214653,4
1,0.547,0.820,8,-6.728,1,0.0805,0.26100,0.000000,0.3000,0.933,118.208,audio_features,2Nz6aF1umHh5Et6I5H581L,spotify:track:2Nz6aF1umHh5Et6I5H581L,https://api.spotify.com/v1/tracks/2Nz6aF1umHh5...,https://api.spotify.com/v1/audio-analysis/2Nz6...,172867,4
2,0.274,0.949,11,-8.642,1,0.0896,0.02140,0.007780,0.2180,0.497,100.346,audio_features,5Q41NLTmGbVPozwHKK7bk2,spotify:track:5Q41NLTmGbVPozwHKK7bk2,https://api.spotify.com/v1/tracks/5Q41NLTmGbVP...,https://api.spotify.com/v1/audio-analysis/5Q41...,406307,4
3,0.500,0.748,4,-9.072,1,0.0363,0.12700,0.000000,0.4470,0.514,118.852,audio_features,4bHsxqR3GMrXTxEPLuK5ue,spotify:track:4bHsxqR3GMrXTxEPLuK5ue,https://api.spotify.com/v1/tracks/4bHsxqR3GMrX...,https://api.spotify.com/v1/audio-analysis/4bHs...,250987,4
4,0.451,0.962,0,-3.947,1,0.0734,0.01880,0.000003,0.2770,0.377,131.435,audio_features,4ECNtOnqzxutZkXP4TE3n3,spotify:track:4ECNtOnqzxutZkXP4TE3n3,https://api.spotify.com/v1/tracks/4ECNtOnqzxut...,https://api.spotify.com/v1/audio-analysis/4ECN...,323707,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,0.496,0.509,7,-15.252,1,0.0326,0.00754,0.005320,0.0610,0.851,92.983,audio_features,4aVs8mfyZjVDivGneIIQ5C,spotify:track:4aVs8mfyZjVDivGneIIQ5C,https://api.spotify.com/v1/tracks/4aVs8mfyZjVD...,https://api.spotify.com/v1/audio-analysis/4aVs...,217427,4
780,0.783,0.878,1,-3.864,0,0.0308,0.39000,0.000778,0.0426,0.885,100.626,audio_features,77oU2rjC5XbjQfNe3bD6so,spotify:track:77oU2rjC5XbjQfNe3bD6so,https://api.spotify.com/v1/tracks/77oU2rjC5Xbj...,https://api.spotify.com/v1/audio-analysis/77oU...,265173,4
781,0.411,0.763,9,-6.738,0,0.0541,0.19900,0.004470,0.3290,0.462,121.570,audio_features,4WFVX3E568hdAixIj3U222,spotify:track:4WFVX3E568hdAixIj3U222,https://api.spotify.com/v1/tracks/4WFVX3E568hd...,https://api.spotify.com/v1/audio-analysis/4WFV...,186560,4
782,0.584,0.708,5,-10.039,1,0.0593,0.02640,0.003640,0.1120,0.564,137.657,audio_features,6iX1f3r7oUJnMbGgQ2gx1j,spotify:track:6iX1f3r7oUJnMbGgQ2gx1j,https://api.spotify.com/v1/tracks/6iX1f3r7oUJn...,https://api.spotify.com/v1/audio-analysis/6iX1...,226200,4


In [60]:
#df2.to_csv('/Users/AFROARC/Documents/GitHub/nomaditect_portfolio/highlighted_projects/song_recomender/python_code/audio_features.csv')

In [ ]:
#merege